In [ ]:
import os
len(os.listdir('/content/drive/MyDrive/coffeAI-detector-first-gen/unsupervised_5_label'))

406

In [ ]:
import os

folder = '/content/drive/MyDrive/coffeAI-detector-first-gen/unsupervised_5_label'

for i in os.listdir(folder):
    arch = folder+'/'+i
    with open(arch, 'r') as f:
        lines = f.readlines()

    # replace each line with the modified label format
    for j in range(len(lines)):
        line = lines[j]
        split_str = line.split()
        split_str[0] = split_str[0].strip('[]')
        lines[j] = ' '.join(split_str) + '\n'

    # overwrite the file with the modified lines
    with open(arch, 'w') as f:
        f.writelines(lines)


In [4]:
import os
import random
import shutil
from tqdm import tqdm


def classes_to_binary(from_folder:str, to_folder:str, map:dict) -> None:
    os.makedirs(to_folder, exist_ok = True)
    for file_dir in os.listdir(from_folder):
        with open(from_folder+'/'+file_dir, 'r') as f:
            new_file_dir = to_folder + '/' + file_dir
            with open(new_file_dir, 'w') as w:
                for line in f:
                    old_line = line 
                    new_line = map[line[0]] + old_line[1:]
                    w.write(new_line)
    return

def count_objects(folder:str) -> None:
    train, val = 0,0 
    for i in os.listdir(folder+'/train/labels/'):
        with open(folder+'/train/labels/' + i, 'r') as f:
            for line in f:
                train += 1
    for j in os.listdir(folder+'/val/labels'):
        with open(folder+'/val/labels/'+j,'r') as r:
            for line in r:
                val += 1
    print('Objects in Train Data: {}\nObjects in Val Data: {}\n'.format(train,val))

def classes_to_mono(from_folder:str, to_folder:str) -> None:
    os.makedirs(to_folder,exist_ok=True)
    for file_dir in os.listdir(from_folder):
        with open(from_folder + '/' + file_dir, 'r') as f:
            new_file_dir = to_folder + '/' + file_dir
            with open(new_file_dir, 'w') as w:
                for line in f:
                    old_line = line
                    new_line = '0'+old_line[1:]
                    w.write(new_line)
    return

def count_objects(folder:str) -> None:
    train, val = 0,0 
    for i in os.listdir(folder+'/train/labels/'):
        with open(folder+'/train/labels/' + i, 'r') as f:
            for line in f:
                train += 1
    for j in os.listdir(folder+'/val/labels'):
        with open(folder+'/val/labels/'+j,'r') as r:
            for line in r:
                val += 1
    print('Objects in Train Data: {}\nObjects in Val Data: {}\n'.format(train,val))

def split_data(folder_path:str, split_percentage:float = 0.8):
    """
    Splits a folder of images and corresponding labels into training and validation sets.

    Args:
        folder_path: Path to the folder containing images and labels.
        split_percentage: Percentage of data to use for training. The remaining percentage will be used for validation.

    Returns:
        train_folder: Path to the folder containing training images and labels.
        val_folder: Path to the folder containing validation images and labels.
    """
    # Create output directories
    train_folder = os.path.join(folder_path, 'train')
    val_folder = os.path.join(folder_path, 'val')
    os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'labels'), exist_ok=True)
    train_folder_imgs,train_folder_lbls = train_folder + '/images/', train_folder + '/labels/'
    val_folder_imgs, val_folder_lbls = val_folder + '/labels/', train_folder + '/labels/'

    # Get list of images
    image_files = [f for f in os.listdir(os.path.join(folder_path, 'images'))]
    num_images = len(image_files)
    num_train = int(num_images * split_percentage)
    num_val = num_images - num_train

    # Shuffle images
    random.shuffle(image_files)

    # Copy images and labels to train folder
    for i in range(num_train):
        image_file = image_files[i]
        label_file = os.path.splitext(image_file)[0] + '.txt'
        src_image = os.path.join(folder_path, 'images', image_file)
        src_label = os.path.join(folder_path, 'labels', label_file)
        dst_image = os.path.join(train_folder, 'images', image_file)
        dst_label = os.path.join(train_folder, 'labels', label_file)
        shutil.copy(src_image, dst_image)
        shutil.copy(src_label, dst_label)

    # Copy images and labels to val folder
    for i in range(num_train, num_train + num_val):
        image_file = image_files[i]
        label_file = os.path.splitext(image_file)[0] + '.txt'
        src_image = os.path.join(folder_path, 'images', image_file)
        src_label = os.path.join(folder_path, 'labels', label_file)
        dst_image = os.path.join(val_folder, 'images', image_file)
        dst_label = os.path.join(val_folder, 'labels', label_file)
        shutil.copy(src_image, dst_image)
        shutil.copy(src_label, dst_label)

    return train_folder, val_folder


def count_imgs_labels(folder:str) -> None:
    train_imgs,val_imgs,train_lbls,val_lbls = 0,0,0,0    
    for i in os.listdir(folder+'/train/images'):
        train_imgs += 1
    for j in os.listdir(folder+'/val/images'): 
        val_imgs += 1 
    for u in os.listdir(folder + '/train/labels'):
        train_lbls += 1 
    for v in os.listdir(folder + '/val/labels'):
        val_lbls += 1
    print('Train Images: {} Train Labels: {}\n\
    Val Images: {} Val Labels: {}'.format(train_imgs,train_lbls, val_imgs,val_lbls))
    print('Total images: {} Total labels: {}'.format(train_imgs+val_imgs, train_lbls+val_lbls))

def yolo_format_to_yolov6(folder:str) -> None:
    imgs_train_folder,imgs_val_folder = folder+'/images/train', folder+'/images/val'
    os.makedirs(imgs_train_folder,exist_ok=True)
    os.makedirs(imgs_val_folder,exist_ok=True)

    lbls_train_folder,lbls_val_folder = folder+'/labels/train', folder+'/labels/val'
    os.makedirs(lbls_train_folder,exist_ok=True)
    os.makedirs(lbls_val_folder,exist_ok=True)

    # Cópia das imagens de treinamento
    for filename in tqdm(os.listdir(folder+'/train/images'), desc='Cópia de imagens de treinamento'):
        shutil.copy(os.path.join(folder, 'train/images', filename), imgs_train_folder)

    # Cópia das imagens de validação
    for filename in tqdm(os.listdir(folder+'/val/images'), desc='Cópia de imagens de validação'):
        shutil.copy(os.path.join(folder, 'val/images', filename), imgs_val_folder)

    # Cópia dos arquivos de labels de treinamento
    for filename in tqdm(os.listdir(folder+'/train/labels'), desc='Cópia de arquivos de labels de treinamento'):
        if filename.endswith('.txt'):
            shutil.copy(os.path.join(folder, 'train/labels', filename), lbls_train_folder)

    # Cópia dos arquivos de labels de validação
    for filename in tqdm(os.listdir(folder+'/val/labels'), desc='Cópia de arquivos de labels de validação'):
        if filename.endswith('.txt'):
            shutil.copy(os.path.join(folder, 'val/labels', filename), lbls_val_folder)

def yolo_format_to_yolov4(folder:str) -> None:
    imgs_train_folder,imgs_val_folder = folder+'/images/train', folder+'/images/val'
    os.makefile('train')


In [ ]:
split_data('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-5-unsup')

('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-5-unsup/train',
 '/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-5-unsup/val')

In [ ]:
count_imgs_labels('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-5-unsup')

Train Images: 324 Train Labels: 324
    Val Images: 82 Val Labels: 82
Total images: 406 Total labels: 406


In [ ]:
count_objects('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-5-unsup')

Objects in Train Data: 35000
Objects in Val Data: 8803



In [ ]:
cd /content/drive/MyDrive/yolov7

/content/drive/MyDrive/yolov7


In [ ]:
!python train.py --epochs 100 --device 0 --batch-size 16 --data data/custom_5_unsup.yaml --img 640 640 --cfg cfg/training/yolov7_5_unsup.yaml --weights 'yolov7.pt' --name yolov7-first-gen-5-unsup 

YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(weights='yolov7.pt', cfg='cfg/training/yolov7_5_unsup.yaml', data='data/custom_5_unsup.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=100, batch_size=16, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='yolov7-first-gen-5-unsup', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/yolov7-first-gen-5-unsup4', total_batch_size=16)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-03-30 22:17:55.628702: I tensorflow/core/platform/cpu_fea

In [2]:
import os
len(os.listdir('/content/drive/MyDrive/coffeAI-detector-first-gen/unsupervised_2_label'))

406

In [3]:
import os

folder = '/content/drive/MyDrive/coffeAI-detector-first-gen/unsupervised_2_label'

for i in os.listdir(folder):
    arch = folder+'/'+i
    with open(arch, 'r') as f:
        lines = f.readlines()

    # replace each line with the modified label format
    for j in range(len(lines)):
        line = lines[j]
        split_str = line.split()
        split_str[0] = split_str[0].strip('[]')
        lines[j] = ' '.join(split_str) + '\n'

    # overwrite the file with the modified lines
    with open(arch, 'w') as f:
        f.writelines(lines)


In [5]:
split_data('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-2-unsup')

('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-2-unsup/train',
 '/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-2-unsup/val')

In [6]:
count_imgs_labels('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-2-unsup')

Train Images: 324 Train Labels: 324
    Val Images: 82 Val Labels: 82
Total images: 406 Total labels: 406


In [7]:
count_objects('/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-2-unsup')

Objects in Train Data: 35264
Objects in Val Data: 8539



In [8]:
cd /content/drive/MyDrive/yolov7

/content/drive/MyDrive/yolov7


In [11]:
!python train.py --epochs 100 --device 0 --batch-size 16 --data data/custom_2_unsup.yaml --img 640 640 --cfg cfg/training/yolov7_2_unsup.yaml --weights 'yolov7.pt' --name yolov7-first-gen-2-unsup 

YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(weights='yolov7.pt', cfg='cfg/training/yolov7_2_unsup.yaml', data='data/custom_2_unsup.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=100, batch_size=16, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='yolov7-first-gen-2-unsup', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/yolov7-first-gen-2-unsup3', total_batch_size=16)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-03-31 17:02:58.629332: I tensorflow/core/platform/cpu_fea

In [1]:
cd /content/drive/MyDrive/yolov7

/content/drive/MyDrive/yolov7


In [2]:
!python detect_continuous.py --device cpu --weights runs/train/yolov7-first-gen-5-unsup4/weights/best.pt --conf 0.5 --img-size 640 --source /content/drive/MyDrive/yolov7/data/coffeAI-firstgen-continuous/val/images

Namespace(weights=['runs/train/yolov7-first-gen-5-unsup4/weights/best.pt'], source='/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-continuous/val/images', img_size=640, conf_thres=0.5, iou_thres=0.45, device='cpu', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36503348 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwar

In [3]:
!python detect_continuous.py --device cpu --weights runs/train/yolov7-first-gen-5-unsup4/weights/best.pt --conf 0.5 --img-size 640 --source /content/drive/MyDrive/yolov7/data/coffeAI-firstgen-continuous/val/images

Namespace(weights=['runs/train/yolov7-first-gen-5-unsup4/weights/best.pt'], source='/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-continuous/val/images', img_size=640, conf_thres=0.5, iou_thres=0.45, device='cpu', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36503348 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwar

In [4]:
!python detect_continuous.py --device cpu --weights /content/drive/MyDrive/yolov7/runs/train/yolov7-first-gen-continuous/weights/best.pt --conf 0.5 --img-size 640 --source /content/drive/MyDrive/yolov7/data/coffeAI-firstgen-continuous/val/images

Namespace(weights=['/content/drive/MyDrive/yolov7/runs/train/yolov7-first-gen-continuous/weights/best.pt'], source='/content/drive/MyDrive/yolov7/data/coffeAI-firstgen-continuous/val/images', img_size=640, conf_thres=0.5, iou_thres=0.45, device='cpu', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36503348 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  ret